# Imports

In [1]:
import numpy as np
import pandas as pd
import pint
from scipy.constants import hbar,c,m_e

from itertools import combinations_with_replacement as cwr

import sys
sys.path.insert(0,'..')
from properties import *

In [2]:
ureg = pint.UnitRegistry()
Q_   = ureg.Quantity

# Problem 3

In [3]:
dx   = Q_(1e-13,'cm')
hbar = Q_(hbar,'J*s')
c    = Q_(c,'m/s')
m_e  = Q_(m_e,'kg')


p = (hbar/2/dx).to('kg*m/s')
print('{:~.1E}'.format(p.to('MeV/m*s')))

E2 = (p**2*c**2 + m_e**2*c**4).to('MeV**2')
E  = (E2)**0.5
print('{:~.1f}'.format(E.to('MeV')))

actual = Q_(0.019,'MeV')

print('%.1f'%(E/actual))

3.3E-07 MeV * s / m
98.7 MeV
5192.9


# Problem 4

In [4]:
bo = baryon_octet().info

# ['p', 'n', 'Lambda^{0}', 'Sigma^{+}', 'Sigma^{-}', 'Sigma^{0}', 'Xi^{-}', 'Xi^{0}']
baryons = list(bo.keys())

N   = baryons[:2]
Xi  = baryons[6:8]
Sig = baryons[3:6]

m_N   = sum(bo[_]['m'] for _ in N)/2
m_Xi  = sum(bo[_]['m'] for _ in Xi)/2
m_Sig = sum(bo[_]['m'] for _ in Sig)/3

print('%.0f'%m_N)
print('%.0f'%m_Xi)
print('%.0f'%m_Sig)

m_Lam = (2*(m_N+m_Xi)-m_Sig)/3
print('%.0f'%m_Lam)

print('%.1f'%(bo['Lambda^{0}']['m']/m_Lam))

939
1318
1193
1107
1.0


# Problem 7

In [10]:
bd = baryon_decuplet().info
bo = baryon_octet().info
mo = meson_octet().info

particles = ['Delta^{-}','Sigma^{*+}','Xi^{*-}']

print('(a)',end='\n')
for particle in particles:
    for baryon in bo.keys():
        for meson in mo.keys():
            Q_cond = bd[particle]['Q'] - (bo[baryon]['Q'] + mo[meson]['Q']) == 0
            S_cond = bd[particle]['S'] - (bo[baryon]['S'] + mo[meson]['S']) == 0

            if Q_cond and S_cond:
                print('\t%s --> %s + %s'%(particle,baryon,meson))
    print()

print(2*'\n')
print('(b)')
for particle in particles:
    for baryon in bo.keys():
        for meson in mo.keys():
            Q_cond = bd[particle]['Q'] - (bo[baryon]['Q'] + mo[meson]['Q']) == 0
            S_cond = bd[particle]['S'] - (bo[baryon]['S'] + mo[meson]['S']) == 0
            M_cond = bd[particle]['m'] - (bo[baryon]['m'] + mo[meson]['m']) >= 0

            if Q_cond and S_cond and M_cond:
                print('\t%s --> %s + %s'%(particle,baryon,meson))
    print()

(a)
	Delta^{-} --> n + pi(-)
	Delta^{-} --> Sigma^{-} + K(0)

	Sigma^{*+} --> p + Kb(0)
	Sigma^{*+} --> Lambda^{0} + pi(+)
	Sigma^{*+} --> Sigma^{+} + pi(0)
	Sigma^{*+} --> Sigma^{+} + eta
	Sigma^{*+} --> Sigma^{0} + pi(+)
	Sigma^{*+} --> Xi^{0} + K(+)

	Xi^{*-} --> Lambda^{0} + K(-)
	Xi^{*-} --> Sigma^{-} + Kb(0)
	Xi^{*-} --> Sigma^{0} + K(-)
	Xi^{*-} --> Xi^{-} + pi(0)
	Xi^{*-} --> Xi^{-} + eta
	Xi^{*-} --> Xi^{0} + pi(-)




(b)
	Delta^{-} --> n + pi(-)

	Sigma^{*+} --> Lambda^{0} + pi(+)
	Sigma^{*+} --> Sigma^{+} + pi(0)
	Sigma^{*+} --> Sigma^{0} + pi(+)

	Xi^{*-} --> Xi^{-} + pi(0)
	Xi^{*-} --> Xi^{0} + pi(-)



# Problem 14

In [7]:
q_info = quark().info
quarks = ['u','d','s','c']
baryons = list(cwr(quarks,3))

table = {'Baryon': [], 'charm': []}
for _ in baryons:
    baryon = ''.join(_)
    table['Baryon'].append(r'$%s$'%baryon)
    table['charm'].append('%.0f'%baryon.count('c'))

table = pd.DataFrame(table)
table.to_latex(buf='prob14.tex',
        index=False,
        escape=False,
        column_format='cc')


# Problem 15

In [8]:
q_info = quark().info
quarks = ['u','d','s','c']
antiquarks = ['ub','db','sb','cb']

table = {'Meson': [], 'charm': []}
for q in quarks:
    for qb in antiquarks:
        table['Meson'].append(r'$%s\bar{%s}$'%(q,qb[0]))
        table['charm'].append((q=='c')-(qb=='cb'))

table = pd.DataFrame(table)
table.to_latex(buf='prob15.tex',
        index=False,
        escape=False,
        column_format='cc')


# Problem 17

In [9]:
m = {'u':336,'d':336,'s':540}
m_bind = -62

bo = baryon_octet().info 

table = {'Baryon ($qqq$)': [],
         'Calculated Mass (\qty{}{\MeV})': [],
         'Percent Error from PDG': []}

table_keys = list(table.keys())
for baryon in bo.keys():
    table[table_keys[0]].append(r'$%s%s~(%s)$'%('\\' if baryon not in ['p','n'] else '',baryon,''.join(bo[baryon]['valence'])))

    temp = sum([m[_] for _ in bo[baryon]['valence']]) + m_bind
    err  = (temp - bo[baryon]['m'])/bo[baryon]['m']*100

    table[table_keys[1]].append('%.0f'%temp)
    table[table_keys[2]].append(r'%.2f\%%'%err)

table = pd.DataFrame(table)
table.to_latex(buf='prob17.tex',
        index=False,
        escape=False,
        column_format='ccc')